# Bitcoin AI trader

## Dependencies

To start make sure all dependencies are installed with this command `pip install numpy tensorflow matplotlib sqlalchemy pandas psycopg2 urllib3 scipy`

## Fetch data

To fetch the data run this command 
* `bash ./download-spot-klines.sh`
* `bash ./unzip-all.sh`
...

In [ ]:
import pandas as pd
import numpy as np
import glob
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, GlobalAveragePooling1D, Concatenate, Dropout, Conv1D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from scipy.stats.mstats import winsorize
from math import ceil
from matplotlib import pyplot as plt
import gc


In [ ]:
def create_optimized_model(action_size, shape):
    input_layer = Input(shape=shape)

    # Convolutional Branch
    conv_branch = Conv1D(filters=64, kernel_size=3, activation='relu')(input_layer)
    conv_branch = BatchNormalization()(conv_branch)
    conv_branch = Dropout(0.3)(conv_branch)
    conv_branch = GlobalAveragePooling1D()(conv_branch)

    # LSTM Branch
    lstm_branch = LSTM(50, return_sequences=True)(input_layer)
    lstm_branch = BatchNormalization()(lstm_branch)
    lstm_branch = Dropout(0.3)(lstm_branch)
    lstm_branch = GlobalAveragePooling1D()(lstm_branch)

    # Combining both branches
    concatenated = Concatenate()([conv_branch, lstm_branch])
    dense = Dense(100, activation='relu', kernel_regularizer=l2(0.01))(concatenated)
    dense = Dropout(0.3)(dense)
    dense = Dense(25, activation='relu', kernel_regularizer=l2(0.01))(dense)
    dense = Dropout(0.2)(dense)
    output = Dense(action_size, activation='sigmoid')(dense)

    model = Model(inputs=input_layer, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
def data_generator(data, actions, rewards, batch_size):
    total_samples = len(data)
    while True:
        for start in range(0, total_samples, batch_size):
            end = min(start + batch_size, total_samples)
            yield data[start:end], actions[start:end], rewards[start:end]

def train_with_accumulation(model, data_gen, steps_per_epoch, accumulation_steps=4, epochs=10):
    optimizer = Adam(learning_rate=0.001)
    loss_fn = tf.keras.losses.BinaryCrossentropy()

    for epoch in range(epochs):
        total_loss = 0
        gradient_accumulation = [tf.zeros_like(v) for v in model.trainable_variables]
        count = 0
        for batch_data, batch_actions, batch_rewards in data_gen:
            with tf.GradientTape() as tape:
                predictions = model(batch_data, training=True)
                loss = loss_fn(batch_rewards, predictions)
            grads = tape.gradient(loss, model.trainable_variables)
            gradient_accumulation = [accum + grad for accum, grad in zip(gradient_accumulation, grads)]
            count += 1

            if count % accumulation_steps == 0 or count == steps_per_epoch:
                optimizer.apply_gradients(zip(gradient_accumulation, model.trainable_variables))
                gradient_accumulation = [tf.zeros_like(v) for v in model.trainable_variables]
                count = 0

            if count == steps_per_epoch:
                break

        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / steps_per_epoch:.4f}")
        gc.collect()

In [ ]:
# Load all CSV files into a single DataFrame
path = 'data/test'  # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    print(f"Processing file: {filename}")
    data = pd.read_csv(filename, index_col=None, header=0)
    if np.any(np.isnan(data)):
        print("NaNs found after importing from csv before conversion to numeric")

    data.columns = [
        'Open time', 'Open', 'High', 'Low', 'Close', 'Volume',
        'Close time', 'Quote asset volume', 'Number of trades',
        'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'
    ]

    if np.any(np.isnan(data)):
        print("NaNs found after renaming columns")

    data['Open time'] = pd.to_datetime(data['Open time'], unit='ms')
    data['Close time'] = pd.to_datetime(data['Close time'], unit='ms')

    if np.any(np.isnan(data)):
        print("NaNs found after conversion to datetime")

    numeric_features = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote asset volume', 'Number of trades',
                        'Taker buy base asset volume', 'Taker buy quote asset volume']

    for feature in ['Open time', 'Close time']:
        data[f'{feature} hour'] = data[feature].dt.hour
        numeric_features.append(f'{feature} hour')
        data[f'{feature} day of week'] = data[feature].dt.weekday
        numeric_features.append(f'{feature} day of week')
        data[f'{feature} week of year'] = data[feature].dt.isocalendar().week.astype(int)
        numeric_features.append(f'{feature} week of year')
        data[f'{feature} month'] = data[feature].dt.month
        # numeric_features.append(f'{feature} month')

    if np.any(np.isnan(data)):
        print("NaNs found after setting time features")

    data[numeric_features] = data[numeric_features].apply(pd.to_numeric, errors='coerce')
    print("After conversion to numeric: ", data[numeric_features].isna().sum())

    if np.any(np.isnan(data)):
        print("NaNs found after conversion to numeric")

    print("Before SMA/LMA: ", data['Close'].isna().sum())
    data['SMA'] = data['Close'].rolling(window=300).mean()
    data['LMA'] = data['Close'].rolling(window=600).mean()
    first_valid_close = data['Close'].dropna().iloc[0]
    data['SMA'] = data['SMA'].fillna(first_valid_close)
    data['LMA'] = data['LMA'].fillna(first_valid_close)
    print("After SMA/LMA: ", data[['SMA', 'LMA']].isna().sum())

    if np.any(np.isnan(data)):
        print("NaNs found after calculating SMA/LMA")

    threshold = 0.01
    data['action'] = np.where(
        (data['SMA'] > data['LMA']) & ((data['SMA'] - data['LMA']) / data['LMA'] > threshold), 1,
        np.where((data['SMA'] < data['LMA']) & ((data['LMA'] - data['SMA']) / data['SMA'] > threshold), -1, 0)
    )

    if np.any(np.isnan(data)):
        print("NaNs found after setting action")

    transaction_cost = 0.0005
    data['reward'] = data['Close'].diff().shift(-1) - (data['Close'] * transaction_cost)
    data['reward'].fillna(0, inplace=True)
    print("After reward: ", data['reward'].isna().sum())

    data[numeric_features] = data[numeric_features].astype(float)

    # Assuming data is now numeric and can be winsorized
    print("Before winsorizing: ", data[numeric_features].isna().sum())
    data[numeric_features] = data[numeric_features].apply(lambda x: winsorize(x, limits=[0.01, 0.01]))
    print("After winsorizing: ", data[numeric_features].isna().sum())

    # Min-max scaling
    min_max_scaler = lambda x: (x - x.min()) / (x.max() - x.min())
    data[numeric_features] = data[numeric_features].apply(min_max_scaler)
    # remove timestamp features
    data.drop(columns=['Open time', 'Close time'], inplace=True)
    features = np.concatenate((numeric_features, ['Open time month', 'Close time month']))
    print("features: ", features)
    print("After scaling: ", data[features].isna().sum())

    if np.any(np.isnan(data)):
        print("NaNs found after preprocessing")

    li.append(data)

combined_data = pd.concat(li, axis=0, ignore_index=True)
actions = combined_data['action'].dropna().astype(int).values
rewards = combined_data['reward'].dropna().astype(float).values

# Assuming data is ready to be transformed and used in model
data = np.expand_dims(combined_data.to_numpy(), axis=0)
actions = np.expand_dims(actions, axis=0)
rewards = np.expand_dims(rewards, axis=0)


In [ ]:

# Plotting Function

def plot_history(history):
    plt.figure(figsize=(10, 5))
    plt.plot(history['loss'], label='Training loss')
    plt.title('Training Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

# Train the model
# model = train_model(model, data, actions, rewards, epochs=10, batch_size=16)
# plot_history(history)
# model.summary()

# Setup for using the data and training the model
# This is an example, modify according to how your actual data is structured and needs to be processed.
# data = np.expand_dims(data, axis=0)
# data = np.repeat(data, 16, axis=0)
steps_per_epoch = ceil(len(data[0]) / 16)

model = create_optimized_model(action_size=3, shape=(data.shape[1], data.shape[2]))
# model.fit(data, actions, batch_size=16, epochs=10)
data = data.astype('float32')
data_gen = data_generator(data, actions, rewards, batch_size=16)
train_with_accumulation(model, data_gen, steps_per_epoch, accumulation_steps=4, epochs=10)


In [ ]:
gc.collect()